# Replication of the optimal strategy on other node


The idea is that nodes have specifities due to their location (whether they are close to a specific type of generator or node).

Therefore, we study how would be the asset profits on other nodes.


In [1]:
import os
import requests
from datetime import datetime, timedelta
from glob import glob
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d 

## download data

In [18]:
selected_nodes_DA = pd.read_csv("../data/part2_selected_nodes_DA.csv")
selected_nodes_DA = selected_nodes_DA.drop(columns=selected_nodes_DA.columns[0])
selected_nodes_RT = pd.read_csv("../data/part2_selected_nodes_RT.csv")
selected_nodes_RT = selected_nodes_RT.drop(columns=selected_nodes_RT.columns[0])
selected_nodes_RT

,MARKET_DAY,NODE,TYPE,VALUE,HE1,HE2,HE3,HE4,HE5,HE6,...,HE15,HE16,HE17,HE18,HE19,HE20,HE21,HE22,HE23,HE24
0,10/1/2022,AMIL.PSGC1.AMP,Gennode,LMP,15.72,20.09,27.47,38.75,41.85,44.41,...,49.06,53.68,48.43,50.93,52.24,46.76,34.17,30.51,-5.31,11.69
1,10/1/2022,EES.NINEMILE4,Gennode,LMP,60.83,40.59,39.74,40.94,43.85,46.33,...,54.44,58.58,52.77,55.43,55.80,49.87,42.46,42.85,42.81,38.67
2,10/1/2022,EES.SAN_JC1_CT,Gennode,LMP,57.03,38.22,37.66,39.19,42.0,44.44,...,51.84,56.21,50.79,53.32,53.74,47.81,40.47,40.25,39.58,36.19
3,10/1/2022,MEC.PPWIND,Gennode,LMP,21.37,17.56,22.91,31.33,36.5,39.35,...,44.43,48.91,44.05,46.74,48.06,41.5,26.21,30.87,18.47,21.15
4,10/1/2022,NSP.NWELOAD,Loadzone,LMP,17.69,13.95,22.03,33.19,39.61,42.65,...,48.88,53.17,47.95,50.86,52.23,46.37,38.11,35.04,22.29,24.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5473,3/31/2025,EES.NINEMILE4,Gennode,LMP,26.27,24.14,23.95,24.30,24.11,27.06,...,31.89,62.83,43.67,43.01,196.30,66.74,49.73,82.25,32.77,28.97
5474,3/31/2025,EES.SAN_JC1_CT,Gennode,LMP,24.41,23.57,23.52,24.56,24.58,27.97,...,31.89,44.34,35.48,37.34,246.87,69.66,51.21,84.2,33.16,28.70
5475,3/31/2025,MEC.PPWIND,Gennode,LMP,13.54,17.43,19.96,20.52,20.6,23.45,...,29.69,29.34,28.31,30.46,193.58,66.42,48.51,80.84,31.97,26.07
5476,3/31/2025,NSP.NWELOAD,Loadzone,LMP,18.81,22.40,25.69,26.75,26.32,29.12,...,31.71,30.91,30.54,31.3,200.14,69.53,52.47,89.8,36.32,30.85


In [19]:
hour_cols = [f'HE{i}' for i in range(1,25)]

# 0. prep the two master LMP tables
selected_nodes_RT_LMP = (
    selected_nodes_RT
    .query("VALUE=='LMP'")
    .assign(
        date   = lambda d: pd.to_datetime(d['MARKET_DAY']),
        Month  = lambda d: d['date'].dt.month
    )
)

selected_nodes_DA_LMP = (
    selected_nodes_DA
    .query("VALUE=='LMP'")
    .assign(
        date   = lambda d: pd.to_datetime(d['MARKET_DAY']),
        Month  = lambda d: d['date'].dt.month
    )
)

In [20]:
nodes = [
    'AMIL.PSGC1.AMP',
    'EES.NINEMILE4',
    'NSP.PRISL1',
    'EES.SAN_JC1_CT',
    'MEC.PPWIND',
    'NSP.NWELOAD'
]

## Define methods and parameters

In [22]:
from scipy.stats import logistic

def fit_logistic(data):
    # 1) force numeric, turn bad tokens into NaN
    y = pd.to_numeric(data['LMP'], errors='coerce').values.astype(float)
    # 2) keep only finite values
    y = y[np.isfinite(y)]

    if len(y) < 2:                       # cannot fit with <2 obs.
        return pd.Series({
            'count'     : len(y),
            'loc'       : np.nan,
            'scale'     : np.nan,
            'mean_level': np.nan,
            'std_level' : np.nan,
        })

    loc, scale = logistic.fit(y)         # MLE
    scale = np.abs(scale)                # ensure scale is positive        
        
    mean_lvl   = loc
    std_lvl    = np.pi * scale / np.sqrt(3)

    return pd.Series({
        'count'     : len(y),
        'loc'       : loc,
        'scale'     : scale,
        'mean_level': mean_lvl,
        'std_level' : std_lvl,
    })

In [ ]:
#  Fixed inputs
n_miners    = 1_000
power_mw    = np.float32(n_miners * 3.25 / 1_000)           # 3.25 MW drawn each hour
btc_per_hour = n_miners * 0.00008 / 24          # BTC produced per hour

btc_price0  = 93_000
sigma_daily = 0.001                              # 1 % daily σ
sigma_hour  = sigma_daily / np.sqrt(24)

hours  =   8760                       # 8 760 for a full year
n_sims = 5_000

# --------------------------------------------
# 1.  Simulate *all* BTC paths at once
# --------------------------------------------
rng     = np.random.default_rng(42) # reproducibility

eps         = rng.normal(0, sigma_hour, size=(n_sims, hours))
btc_paths   = btc_price0 * np.exp(eps).cumprod(axis=1)     # (n_sims,hours)


# build hourly index for Jul 1 2025 → Jun 30 2026
range = pd.date_range('2025-07-01', '2026-06-30 23:00', freq='h')
sim_df = pd.DataFrame({'DT': range})
sim_df['Month'] = sim_df.DT.dt.month
sim_df['Hour']  = sim_df.DT.dt.hour + 1         # HE1–HE24


### Repeat part 1 on all nodes

In [29]:
results = []

rng_btc = np.random.default_rng(42)
eps     = rng_btc.normal(0, sigma_hour, size=(n_sims, hours))
btc_paths = btc_price0 * np.exp(eps).cumprod(axis=1)

for node in nodes:
    print(f"Processing node: {node}")
    rt = selected_nodes_RT_LMP[selected_nodes_RT_LMP['NODE']==node]
    da = selected_nodes_DA_LMP[selected_nodes_DA_LMP['NODE']==node]
    
    # reshape to long, extracting the hour
    DA_long = (
        da
        .melt(['date','Month'], hour_cols, var_name='HE', value_name='LMP')
        .assign(Hour=lambda d: d['HE'].str.extract(r'HE(\d+)').astype(int))
        .drop(columns='HE')
        .dropna(subset=['LMP'])
    )
    RT_long = (
        rt
        .melt(['date','Month'], hour_cols, var_name='HE', value_name='LMP')
        .assign(Hour=lambda d: d['HE'].str.extract(r'HE(\d+)').astype(int))
        .drop(columns='HE')
        .dropna(subset=['LMP'])
    )

    # now fit logistic, merge into sim_df, draw prices & compute P&L exactly as before
    lp_DA = DA_long.groupby(['Month','Hour']).apply(fit_logistic).reset_index()
    lp_RT = RT_long.groupby(['Month','Hour']).apply(fit_logistic).reset_index()

    sim  = sim_df.copy()
    sim_DA = sim.merge(lp_DA[['Month','Hour','loc','scale']], on=['Month','Hour'], how='left')
    sim_RT = sim.merge(lp_RT[['Month','Hour','loc','scale']], on=['Month','Hour'], how='left')

    P_da = rng_btc.logistic(sim_DA['loc'], sim_DA['scale'], size=(n_sims,hours))
    P_rt = rng_btc.logistic(sim_RT['loc'], sim_RT['scale'], size=(n_sims,hours))

    prof_da = (btc_per_hour*btc_paths - power_mw*P_da).sum(axis=1)
    prof_rt = (btc_per_hour*btc_paths - power_mw*P_rt).sum(axis=1)

    rev_mine  = btc_paths * btc_per_hour
    cost_mine = power_mw   * P_da
    prof1     = rev_mine - cost_mine
    prof2     = power_mw*(P_rt - P_da)
    neg_prof1 = prof1 < 0
    best      = np.maximum(prof1, prof2)
    best[neg_prof1] = 0  # set negative profits to zero
    prof_ot   = best.sum(axis=1)
    opt1_h    = (prof1>prof2).sum(axis=1)
    opt2_h    = hours - opt1_h

    results.append({
      'NODE':     node,
      'Mean_DA':  prof_da.mean(),  'Std_DA':  prof_da.std(),
      'Mean_RT':  prof_rt.mean(),  'Std_RT':  prof_rt.std(),
      'Mean_OT':  prof_ot.mean(),  'Std_OT':  prof_ot.std(),
      #'AvgOpt1H': opt1_h.mean(),   'AvgOpt2H': opt2_h.mean(),
    })

summary = pd.DataFrame(results)

Processing node: AMIL.PSGC1.AMP


C:\Users\benja\AppData\Local\Temp\ipykernel_26952\1945896658.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lp_DA = DA_long.groupby(['Month','Hour']).apply(fit_logistic).reset_index()
C:\Users\benja\AppData\Local\Temp\ipykernel_26952\1945896658.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lp_RT = RT_long.groupby(['Month','Hour']).apply(fit_logistic).reset_index()


Processing node: EES.NINEMILE4


C:\Users\benja\AppData\Local\Temp\ipykernel_26952\1945896658.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lp_DA = DA_long.groupby(['Month','Hour']).apply(fit_logistic).reset_index()
C:\Users\benja\AppData\Local\Temp\ipykernel_26952\1945896658.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lp_RT = RT_long.groupby(['Month','Hour']).apply(fit_logistic).reset_index()


Processing node: NSP.PRISL1


...

C:\Users\benja\AppData\Local\Temp\ipykernel_26952\1945896658.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lp_DA = DA_long.groupby(['Month','Hour']).apply(fit_logistic).reset_index()
C:\Users\benja\AppData\Local\Temp\ipykernel_26952\1945896658.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lp_RT = RT_long.groupby(['Month','Hour']).apply(fit_logistic).reset_index()


Processing node: MEC.PPWIND


C:\Users\benja\AppData\Local\Temp\ipykernel_26952\1945896658.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lp_DA = DA_long.groupby(['Month','Hour']).apply(fit_logistic).reset_index()
C:\Users\benja\AppData\Local\Temp\ipykernel_26952\1945896658.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lp_RT = RT_long.groupby(['Month','Hour']).apply(fit_logistic).reset_index()


Processing node: NSP.NWELOAD


C:\Users\benja\AppData\Local\Temp\ipykernel_26952\1945896658.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lp_DA = DA_long.groupby(['Month','Hour']).apply(fit_logistic).reset_index()
C:\Users\benja\AppData\Local\Temp\ipykernel_26952\1945896658.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lp_RT = RT_long.groupby(['Month','Hour']).apply(fit_logistic).reset_index()


In [30]:
# 1. compute Sharpe ratios
summary['Sharpe_DA'] = summary['Mean_DA'] / summary['Std_DA']
summary['Sharpe_RT'] = summary['Mean_RT'] / summary['Std_RT']
summary['Sharpe_OT'] = summary['Mean_OT'] / summary['Std_OT']

# 2. create a styled DataFrame with $ + commas, no scientific notation
fmt = {
    'Mean_DA'  : '${:,.0f}', 'Std_DA'  : '${:,.0f}',
    'Mean_RT'  : '${:,.0f}', 'Std_RT'  : '${:,.0f}',
    'Mean_OT'  : '${:,.0f}', 'Std_OT'  : '${:,.0f}',
    'Sharpe_DA': '{:.2f}',   'Sharpe_RT': '{:.2f}',
    'Sharpe_OT': '{:.2f}',
    'AvgOpt1H' : '{:,.1f}',  'AvgOpt2H': '{:,.1f}'
}

summary = summary.style.format(fmt)

summary

,NODE,Mean_DA,Std_DA,Mean_RT,Std_RT,Mean_OT,Std_OT,Sharpe_DA,Sharpe_RT,Sharpe_OT
0,AMIL.PSGC1.AMP,"$1,879,309","$29,961","$1,947,369","$30,073","$1,885,932","$29,676",62.73,64.76,63.55
1,EES.NINEMILE4,"$1,877,082","$29,836","$1,939,706","$29,932","$1,880,569","$29,650",62.91,64.80,63.43
2,NSP.PRISL1,"$1,978,217","$29,828","$2,039,817","$30,151","$1,983,643","$29,600",66.32,67.65,67.02
3,EES.SAN_JC1_CT,"$1,893,177","$29,824","$1,955,701","$30,065","$1,898,321","$29,604",63.48,65.05,64.12
4,MEC.PPWIND,"$2,165,346","$30,198","$2,221,185","$49,441","$2,250,529","$37,177",71.71,44.93,60.54
5,NSP.NWELOAD,"$1,835,564","$29,947","$1,918,088","$30,350","$1,846,849","$29,491",61.29,63.20,62.63


$\textbf{Commentary}: $ Even tough the optimal strategy does not always have the best sharpe ratio, we see that it provides consistent metrics across all nodes (which is no the case for the naive strategies). This is a good thing, when investing, to have consistent performence.

However, probably that when investing on a specific node, doing more research on the price statistics could lead to higher performance

## Adding a node or doubling the size?


In [32]:
node_RT_original = pd.read_csv("../data/Node_RT.csv")
node_DA_original = pd.read_csv("../data/Node_DA_LMP.csv")

node_RT_LMP_original = node_RT_original[node_RT_original['VALUE'] == 'LMP']
node_RT_LMP_original = node_RT_LMP_original.drop(columns=node_RT_LMP_original.columns[0])#.set_index('MARKET_DAY')
node_RT_LMP_original['date'] = pd.to_datetime(node_RT_LMP_original['MARKET_DAY']) 
node_RT_LMP_original['Month'] = node_RT_LMP_original['date'].dt.month
node_DA_LMP_original = node_DA_original
node_DA_LMP_original['date'] = pd.to_datetime(node_DA_LMP_original['MARKET_DAY'])
node_DA_LMP_original['Month'] = node_DA_LMP_original['date'].dt.month
node_DA_LMP_original.drop(columns=["source_zip"], inplace=True)

In [33]:
DA_long_original  = (
    node_DA_LMP_original
    .melt(id_vars=['date', 'Month'], value_vars=hour_cols,
          var_name='Hour', value_name='LMP')
    .assign(Hour=lambda d: d['Hour'].str.extract(r'HE(\d+)').astype(int))
    .dropna(subset=['LMP'])
)

RT_long_original = (
    node_RT_LMP_original
    .melt(id_vars=['date', 'Month'], value_vars=hour_cols,
          var_name='Hour', value_name='LMP')
    .assign(Hour=lambda d: d['Hour'].str.extract(r'HE(\d+)').astype(int))
    .dropna(subset=['LMP'])
)

In [34]:
# ── Apply to DA and RT data ──────────────────────────────────────────
logistic_params_DA_original = (
    DA_long_original.groupby(['Month', 'Hour'])
           .apply(fit_logistic)
           .reset_index()
)

logistic_params_RT_original = (
    RT_long_original.groupby(['Month', 'Hour'])
           .apply(fit_logistic)
           .reset_index()
)

sim_df_DA_original = sim_df.merge(logistic_params_DA_original[['mean_level','scale','std_level','Month','Hour']], on=['Month','Hour'], how='left')
sim_df_RT_original = sim_df.merge(logistic_params_RT_original[['mean_level','scale','std_level','Month','Hour']], on=['Month','Hour'], how='left')

C:\Users\benja\AppData\Local\Temp\ipykernel_26952\2962648125.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(fit_logistic)
C:\Users\benja\AppData\Local\Temp\ipykernel_26952\2962648125.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(fit_logistic)


### apply optimal strategy to original node to get hourly pnl

In [38]:
def logistic_draw(mean, scale):
    loc   = mean.values.astype('float32')  
    scale_val = scale.values.astype('float32')  
    return rng.logistic(loc, scale_val, size=(n_sims, hours))   # (n_sims,hours)

rng     = np.random.default_rng(42) # reproducibility

P_rt_original = logistic_draw(sim_df_RT_original['mean_level'], sim_df_RT_original['scale'])
P_da_original = logistic_draw(sim_df_DA_original['mean_level'], sim_df_DA_original['scale'])

#  Compute both profit options per hour
rev_mine  = btc_paths * btc_per_hour                # $ from mining
cost_powd_da  = power_mw * P_da_original                         # $ cost of power in DA to mine
cost_powd_rt  = power_mw * P_rt_original                         # $ cost of power in RT to mine

prof1 = rev_mine - cost_powd_da                         # mining option
prof2 = power_mw * (P_rt_original - P_da_original)                    # sell DA power back in RT market

## Da strat
pnl_da_original = prof1.sum(axis=1)  # (n_sims,)
#  RT strat
pnl_rt_original = (rev_mine - cost_powd_rt).sum(axis=1)  # (n_sims,)

#  Choose best option per hour & accumulate
best_pnl   = np.maximum(prof1, prof2)               # (n_sims,hours)
best_pnl[prof1 < 0] = 0                            # set negative profits to zero
profits_OT_original = best_pnl.sum(axis=1)                   # (n_sims,)


## Now we consider having 2 nodes: Original + another

Therefore we sum the hourly pnl and compute the statistics on that

In [47]:
results_2_nodes = []

rng_btc = np.random.default_rng(42)
eps     = rng_btc.normal(0, sigma_hour, size=(n_sims, hours))
btc_paths = btc_price0 * np.exp(eps).cumprod(axis=1)

for node in nodes:
    print(f"Processing node: {node}")
    rt = selected_nodes_RT_LMP[selected_nodes_RT_LMP['NODE']==node]
    da = selected_nodes_DA_LMP[selected_nodes_DA_LMP['NODE']==node]
    
    # reshape to long, extracting the hour
    DA_long = (
        da
        .melt(['date','Month'], hour_cols, var_name='HE', value_name='LMP')
        .assign(Hour=lambda d: d['HE'].str.extract(r'HE(\d+)').astype(int))
        .drop(columns='HE')
        .dropna(subset=['LMP'])
    )
    RT_long = (
        rt
        .melt(['date','Month'], hour_cols, var_name='HE', value_name='LMP')
        .assign(Hour=lambda d: d['HE'].str.extract(r'HE(\d+)').astype(int))
        .drop(columns='HE')
        .dropna(subset=['LMP'])
    )

    # now fit logistic, merge into sim_df, draw prices & compute P&L exactly as before
    lp_DA = DA_long.groupby(['Month','Hour']).apply(fit_logistic).reset_index()
    lp_RT = RT_long.groupby(['Month','Hour']).apply(fit_logistic).reset_index()

    sim  = sim_df.copy()
    sim_DA = sim.merge(lp_DA[['Month','Hour','loc','scale']], on=['Month','Hour'], how='left')
    sim_RT = sim.merge(lp_RT[['Month','Hour','loc','scale']], on=['Month','Hour'], how='left')

    P_da = rng_btc.logistic(sim_DA['loc'], sim_DA['scale'], size=(n_sims,hours))
    P_rt = rng_btc.logistic(sim_RT['loc'], sim_RT['scale'], size=(n_sims,hours))

    prof_da = (btc_per_hour*btc_paths - power_mw*P_da).sum(axis=1)
    prof_da_2_nodes = prof_da + pnl_da_original
    prof_rt = (btc_per_hour*btc_paths - power_mw*P_rt).sum(axis=1)
    prof_rt_2_nodes = prof_rt + pnl_rt_original

    rev_mine  = btc_paths * btc_per_hour
    cost_mine = power_mw   * P_da
    prof1     = rev_mine - cost_mine
    prof2     = power_mw*(P_rt - P_da)
    neg_prof1 = prof1 < 0
    best      = np.maximum(prof1, prof2)
    best[neg_prof1] = 0  # set negative profits to zero
    prof_ot   = best.sum(axis=1)
    prof_OT_2_nodes = profits_OT_original + prof_ot

    results_2_nodes.append({
      'NODE':     node,
      'Mean_DA':  prof_da_2_nodes.mean(),  'Std_DA':  prof_da_2_nodes.std(),
      'Mean_RT':  prof_rt_2_nodes.mean(),  'Std_RT':  prof_rt_2_nodes.std(),
      'Mean_OT':  prof_OT_2_nodes.mean(),  'Std_OT':  prof_OT_2_nodes.std(),
      #'AvgOpt1H': opt1_h.mean(),   'AvgOpt2H': opt2_h.mean(),
    })

summary2 = pd.DataFrame(results_2_nodes)

Processing node: AMIL.PSGC1.AMP


C:\Users\benja\AppData\Local\Temp\ipykernel_26952\1549991044.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lp_DA = DA_long.groupby(['Month','Hour']).apply(fit_logistic).reset_index()
C:\Users\benja\AppData\Local\Temp\ipykernel_26952\1549991044.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lp_RT = RT_long.groupby(['Month','Hour']).apply(fit_logistic).reset_index()


Processing node: EES.NINEMILE4


C:\Users\benja\AppData\Local\Temp\ipykernel_26952\1549991044.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lp_DA = DA_long.groupby(['Month','Hour']).apply(fit_logistic).reset_index()
C:\Users\benja\AppData\Local\Temp\ipykernel_26952\1549991044.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lp_RT = RT_long.groupby(['Month','Hour']).apply(fit_logistic).reset_index()


Processing node: NSP.PRISL1


...

C:\Users\benja\AppData\Local\Temp\ipykernel_26952\1549991044.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lp_DA = DA_long.groupby(['Month','Hour']).apply(fit_logistic).reset_index()
C:\Users\benja\AppData\Local\Temp\ipykernel_26952\1549991044.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lp_RT = RT_long.groupby(['Month','Hour']).apply(fit_logistic).reset_index()


Processing node: MEC.PPWIND


C:\Users\benja\AppData\Local\Temp\ipykernel_26952\1549991044.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lp_DA = DA_long.groupby(['Month','Hour']).apply(fit_logistic).reset_index()
C:\Users\benja\AppData\Local\Temp\ipykernel_26952\1549991044.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lp_RT = RT_long.groupby(['Month','Hour']).apply(fit_logistic).reset_index()


Processing node: NSP.NWELOAD


C:\Users\benja\AppData\Local\Temp\ipykernel_26952\1549991044.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lp_DA = DA_long.groupby(['Month','Hour']).apply(fit_logistic).reset_index()
C:\Users\benja\AppData\Local\Temp\ipykernel_26952\1549991044.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lp_RT = RT_long.groupby(['Month','Hour']).apply(fit_logistic).reset_index()


In [48]:
# 1. compute Sharpe ratios
rf = 0.04 
initial_investment_2 = 2*500_000  # Initial investment in USD
rf_return_2 = initial_investment_2 * rf  # Risk-free return over the period
summary2['Sharpe_DA'] = (summary2['Mean_DA']-rf_return_2) / summary2['Std_DA']
summary2['Sharpe_RT'] = (summary2['Mean_RT']-rf_return_2) / summary2['Std_RT']
summary2['Sharpe_OT'] = (summary2['Mean_OT']-rf_return_2) / summary2['Std_OT']

# 2. create a styled DataFrame with $ + commas, no scientific notation
fmt = {
    'Mean_DA'  : '${:,.0f}', 'Std_DA'  : '${:,.0f}',
    'Mean_RT'  : '${:,.0f}', 'Std_RT'  : '${:,.0f}',
    'Mean_OT'  : '${:,.0f}', 'Std_OT'  : '${:,.0f}',
    'Sortino_DA': '{:.2f}',   'Sortino_RT': '{:.2f}',
    'Sortino_OT': '{:.2f}',
    'AvgOpt1H' : '{:,.1f}',  'AvgOpt2H': '{:,.1f}'
}
summary2.rename(columns={'NODE': 'Orginal Node + Node'}, inplace=True)
summary2 = summary2.style.format(fmt)

summary2

,Orginal Node + Node,Mean_DA,Std_DA,Mean_RT,Std_RT,Mean_OT,Std_OT,Sharpe_DA,Sharpe_RT,Sharpe_OT
0,AMIL.PSGC1.AMP,"$3,714,922","$59,667","$3,865,547","$59,721","$3,732,808","$58,914",61.590817,64.057404,62.681252
1,EES.NINEMILE4,"$3,712,695","$59,570","$3,857,884","$59,660","$3,727,446","$58,919",61.653480,63.994042,62.585312
2,NSP.PRISL1,"$3,813,830","$59,528","$3,957,995","$59,771","$3,830,520","$58,836",63.395359,65.550617,64.425562
3,EES.SAN_JC1_CT,"$3,728,790","$59,542","$3,873,879","$59,762","$3,745,198","$58,859",61.953059,64.152122,62.950635
4,MEC.PPWIND,"$4,000,959","$59,737","$4,139,363","$71,837","$4,097,406","$62,937",66.306842,57.064980,64.467485
5,NSP.NWELOAD,"$3,671,178","$59,607","$3,836,266","$59,896","$3,693,726","$58,685",60.918971,63.381392,62.260211


Comparing with Part 1, we see that through diversification (of nodes), we always achieve higher ratios than just doubling the investment at the original node. We see that for some nodes there is almost no improvement, whereas for other we achieve a slight better performence (recall the Sortino ratio for 2 original nodes = Sortino Ratio for 1 original node = 61.93)